In [10]:
from datasets import load_metric, load_dataset, Audio
from transformers import Wav2Vec2ForCTC, AutoProcessor
import torch
from tqdm import tqdm

In [2]:
# Hausa
cache_dir="/data/users/kashrest/lrl-asr-experiments/data/fleurs"
stream_data = load_dataset("google/fleurs", "ha_ng", split="test", cache_dir=cache_dir, streaming=True)
sample = next(iter(stream_data))
ha_sample = sample["audio"]["array"]
ha_sample_transcription = sample["transcription"]

In [35]:
# Hausa
from huggingface_hub import notebook_login

notebook_login()

cache_dir="/data/users/kashrest/lrl-asr-experiments/data/cv_13"

data = load_dataset("mozilla-foundation/common_voice_13_0", "ha", split="test", cache_dir=cache_dir)
data = data.cast_column("audio", Audio(sampling_rate=16_000))

Using the latest cached version of the module from /data/users/kashrest/huggingface/modules/datasets_modules/datasets/mozilla-foundation--common_voice_13_0/22809012aac1fc9803eaffc44122e4149043748e93933935d5ea19898587e4d7 (last modified on Sun May 21 19:02:16 2023) since it couldn't be found locally at mozilla-foundation/common_voice_13_0., or remotely on the Hugging Face Hub.
Found cached dataset common_voice_13_0 (/data/users/kashrest/lrl-asr-experiments/data/cv_13/mozilla-foundation___common_voice_13_0/ha/13.0.0/22809012aac1fc9803eaffc44122e4149043748e93933935d5ea19898587e4d7)


In [18]:
data[0]

{'id': 1801,
 'num_samples': 303360,
 'path': '/data/users/kashrest/asr-experiments/downloads/extracted/fac2676a78ab0636b6b11f8ff4da4f6e98ba2d10be2c0aa7f2b7c1a89819b928/10063376879559181525.wav',
 'audio': {'path': 'test/10063376879559181525.wav',
  'array': array([0.        , 0.        , 0.        , ..., 0.00063068, 0.00077605,
         0.00100565]),
  'sampling_rate': 16000},
 'transcription': 'an kwatanta faretin gine-ginen da ke yin sararin samaniyar hong kong da ginshiƙi mai walƙiya wanda aka bayyana ta gaban ruwan victoria harbor',
 'raw_transcription': 'An kwatanta faretin gine-ginen da ke yin sararin samaniyar Hong Kong da ginshiƙi mai walƙiya wanda aka bayyana ta gaban ruwan Victoria Harbor.',
 'gender': 1,
 'lang_id': 30,
 'language': 'Hausa',
 'lang_group_id': 3}

In [ ]:
data_samples = []


In [3]:
model_id = "facebook/mms-1b-all"

processor = AutoProcessor.from_pretrained(model_id)
model = Wav2Vec2ForCTC.from_pretrained(model_id)

In [4]:
processor.tokenizer.set_target_lang("hau")
model.load_adapter("hau")

inputs = processor(ha_sample, sampling_rate=16_000, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs).logits

ids = torch.argmax(outputs, dim=-1)[0]
transcription = processor.decode(ids)
# 'wachambuzi wa soka wanamtaja mesi kama nyota hatari zaidi duniani'
# => In English: "soccer analysts describe Messi as the most dangerous player in the world"

In [15]:
processor.tokenizer.get_vocab()["hau"]

{'!': 74,
 '$': 52,
 '%': 56,
 '&': 76,
 "'": 28,
 '+': 64,
 ',': 45,
 '-': 33,
 '.': 43,
 '/': 48,
 '0': 30,
 '1': 32,
 '2': 34,
 '3': 40,
 '4': 39,
 '5': 38,
 '6': 42,
 '7': 44,
 '8': 41,
 '9': 36,
 ':': 50,
 ';': 54,
 '<': 65,
 '</s>': 2,
 '<pad>': 0,
 '<s>': 1,
 '<unk>': 3,
 '>': 66,
 '[': 62,
 ']': 63,
 'a': 4,
 'b': 19,
 'c': 23,
 'd': 12,
 'e': 16,
 'f': 24,
 'g': 21,
 'h': 20,
 'i': 7,
 'j': 26,
 'k': 8,
 'l': 22,
 'm': 14,
 'n': 6,
 'o': 18,
 'p': 31,
 'q': 49,
 'r': 10,
 's': 11,
 't': 15,
 'u': 9,
 'v': 35,
 'w': 17,
 'x': 46,
 'y': 13,
 'z': 25,
 '|': 5,
 '£': 71,
 '¥': 68,
 '°': 61,
 '²': 72,
 'á': 58,
 'ã': 70,
 'ç': 67,
 'é': 69,
 'í': 59,
 'ó': 73,
 'õ': 60,
 'ö': 78,
 'ú': 75,
 'ü': 53,
 'ā': 77,
 'ă': 82,
 'ū': 83,
 'ƙ': 27,
 'ƴ': 55,
 'ɓ': 37,
 'ɗ': 29,
 'ʻ': 79,
 'ˈ': 81,
 '—': 80,
 '‘': 57,
 '’': 47,
 '”': 51}

In [17]:
from finetuning_util_hausa import preprocess_text

In [36]:
#batch = processor.pad(input_features, padding=self.padding, max_length=self.max_length, pad_to_multiple_of=self.pad_to_multiple_of, return_tensors="pt")
processor.tokenizer.set_target_lang("hau")
model.load_adapter("hau")
transcriptions = []
gold_transcriptions = []
for elem in tqdm(data):
    inputs = processor(elem["audio"]["array"], sampling_rate=16_000, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs).logits
    ids = torch.argmax(outputs, dim=-1)[0]
    transcriptions.append(processor.decode(ids))
    gold_transcriptions.append(preprocess_text(elem["sentence"]))
    # 'wachambuzi wa soka wanamtaja mesi kama nyota hatari zaidi duniani'
    # => In English: "soccer analysts describe Messi as the most dangerous player in the world"

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 659/659 [05:24<00:00,  2.03it/s]


In [37]:
len(transcriptions), len(gold_transcriptions)

(659, 659)

In [38]:
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")
wer = wer_metric.compute(predictions=transcriptions, references=gold_transcriptions)
cer = cer_metric.compute(predictions=transcriptions, references=gold_transcriptions)
wer, cer

(0.3554274735830932, 0.07887218045112782)

In [39]:
transcriptions[0]

'habibu da hamsatu ba su kyauta min ba'

In [40]:
gold_transcriptions[0]

'habibu da hamsatu ba su kyauta min ba.'